In [1]:
#Based off of TM-Evaluate-Models-Worksheet in that instead of getting a score for a test set we are printing bets for 
#a test set

In [2]:
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

remove_fight_island = False    #Removes fight island fights from consideration

In [3]:
import sys
sys.path.append('../automated_model_creation') #We need to access the function file

In [4]:
import pandas as pd
import numpy as np
from functions import custom_cv_eval, get_ev_from_df, get_bet_ev, get_bet_return
import random
import csv


In [5]:
df = pd.read_csv("../data/kaggle_data/ufc-master.csv")
df_upcoming = pd.read_csv('../data/kaggle_data/upcoming-event.csv')
len(df)





4783

In [6]:
#The fight island fights were messing with the established models.  This function most likely isn't needed anymore
if remove_fight_island:
    ##Let's remove the Fight island contests and see how that affects score
    df_no_fight_island = df[(df['location'] != 'Abu Dhabi, Abu Dhabi, United Arab Emirates')]
    #df_train_no_fight_island = df_train[(df_train['location'] != 'Abu Dhabi, Abu Dhabi, United Arab Emirates')]
    df = df_no_fight_island
    #df_train = df_train_no_fight_island

len(df)

4783

In [7]:
num_upcoming_fights = len(df_upcoming)

In [8]:
df = df_upcoming.append(df)
len(df)

4794

In [9]:
#Let's put all the labels in a dataframe
df['label'] = ''
#If the winner is not Red or Blue we can remove it.
mask = df['Winner'] == 'Red'
df['label'][mask] = 0
mask = df['Winner'] == 'Blue'
df['label'][mask] = 1

#df["Winner"] = df["Winner"].astype('category')
#df = df[(df['Winner'] != 'Blue') | (df['Winner'] == 'Red') ]


#Make sure lable is numeric
df['label'] = pd.to_numeric(df['label'], errors='coerce')

<ipython-input-9-e6abcb5e0e70>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['label'][mask] = 0
<ipython-input-9-e6abcb5e0e70>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['label'][mask] = 1


In [10]:
#Let's fix the date
df['date'] = pd.to_datetime(df['date'])

In [11]:
#Create a label df:
label_df = df['label']

#Let's create an odds df too:
odds_df = df[['R_odds', 'B_odds']]

In [12]:
#Split the test set.  We are always(?) going to use the last 200 matches as the test set, so we don't want those around
#as we pick models

df_train = df[num_upcoming_fights:]
odds_train = odds_df[num_upcoming_fights:]
label_train = label_df[num_upcoming_fights:]

df_test = df[:num_upcoming_fights]
odds_test = odds_df[:num_upcoming_fights]
label_test = label_df[:num_upcoming_fights]

print(len(df_test))
print(len(odds_test))
print(len(label_test))

print(len(df_train))
print(len(odds_train))
print(len(label_train))

11
11
11
4783
4783
4783


In [13]:
weightclass_list = ['B_match_weightclass_rank', 'R_match_weightclass_rank', "R_Women's Flyweight_rank", "R_Women's Featherweight_rank", "R_Women's Strawweight_rank", "R_Women's Bantamweight_rank", 'R_Heavyweight_rank', 'R_Light Heavyweight_rank', 'R_Middleweight_rank', 'R_Welterweight_rank', 'R_Lightweight_rank', 'R_Featherweight_rank', 'R_Bantamweight_rank', 'R_Flyweight_rank', 'R_Pound-for-Pound_rank', "B_Women's Flyweight_rank", "B_Women's Featherweight_rank", "B_Women's Strawweight_rank", "B_Women's Bantamweight_rank", 'B_Heavyweight_rank', 'B_Light Heavyweight_rank', 'B_Middleweight_rank', 'B_Welterweight_rank', 'B_Lightweight_rank', 'B_Featherweight_rank', 'B_Bantamweight_rank', 'B_Flyweight_rank', 'B_Pound-for-Pound_rank']
df_train[weightclass_list] = df_train[weightclass_list].fillna(17)
df_test[weightclass_list] = df_test[weightclass_list].fillna(17)

C:\Users\matth\anaconda3\lib\site-packages\pandas\core\frame.py:3191: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [14]:
display(df_train)
display(df_test)

,R_fighter,B_fighter,R_odds,B_odds,R_ev,B_ev,date,location,country,Winner,...,B_pass_bout,R_rev_bout,B_rev_bout,r_dec_odds,b_dec_odds,r_sub_odds,b_sub_odds,r_ko_odds,b_ko_odds,label
0,Dustin Poirier,Conor McGregor,-129,100,77.519380,100.000000,2021-07-10,"Las Vegas, Nevada, USA",USA,Red,...,NaN,NaN,NaN,550.0,700.0,575.0,2200.0,165.0,150.0,0.0
1,Gilbert Burns,Stephen Thompson,130,-162,130.000000,61.728395,2021-07-10,"Las Vegas, Nevada, USA",USA,Red,...,NaN,NaN,NaN,400.0,150.0,475.0,2500.0,525.0,250.0,0.0
2,Tai Tuivasa,Greg Hardy,-130,105,76.923077,105.000000,2021-07-10,"Las Vegas, Nevada, USA",USA,Red,...,NaN,NaN,NaN,350.0,350.0,1600.0,1800.0,160.0,265.0,0.0
3,Irene Aldana,Yana Kunitskaya,-107,-115,93.457944,86.956522,2021-07-10,"Las Vegas, Nevada, USA",USA,Red,...,NaN,NaN,NaN,160.0,150.0,1200.0,1200.0,425.0,800.0,0.0
4,Sean O'Malley,Kris Moutinho,-1000,580,10.000000,580.000000,2021-07-10,"Las Vegas, Nevada, USA",USA,Red,...,NaN,NaN,NaN,400.0,1400.0,800.0,1600.0,-300.0,1200.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4778,Duane Ludwig,Darren Elkins,-155,135,64.516129,135.000000,2010-03-21,"Broomfield, Colorado, USA",USA,Blue,...,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0
4779,John Howard,Daniel Roberts,-210,175,47.619048,175.000000,2010-03-21,"Broomfield, Colorado, USA",USA,Red,...,1.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0
4780,Brendan Schaub,Chase Gormley,-260,220,38.461538,220.000000,2010-03-21,"Broomfield, Colorado, USA",USA,Red,...,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0
4781,Mike Pierce,Julio Paulino,-420,335,23.809524,335.000000,2010-03-21,"Broomfield, Colorado, USA",USA,Red,...,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0


,R_fighter,B_fighter,R_odds,B_odds,R_ev,B_ev,date,location,country,Winner,...,B_pass_bout,R_rev_bout,B_rev_bout,r_dec_odds,b_dec_odds,r_sub_odds,b_sub_odds,r_ko_odds,b_ko_odds,label
0,Islam Makhachev,Thiago Moises,-625,425,16.000000,425.000000,2021-07-17,"Las Vegas, Nevada, USA",USA,NaN,...,NaN,NaN,NaN,-120.0,800.0,575.0,700.0,350.0,1200.0,NaN
1,Marion Reneau,Miesha Tate,105,-134,105.000000,74.626866,2021-07-17,"Las Vegas, Nevada, USA",USA,NaN,...,NaN,NaN,NaN,185.0,115.0,475.0,850.0,1100.0,1600.0,NaN
2,Jeremy Stephens,Mateusz Gamrot,195,-250,195.000000,40.000000,2021-07-17,"Las Vegas, Nevada, USA",USA,NaN,...,NaN,NaN,NaN,460.0,245.0,NaN,NaN,NaN,NaN,NaN
3,Rodolfo Vieira,Dustin Stoltzfus,-250,195,40.000000,195.000000,2021-07-17,"Las Vegas, Nevada, USA",USA,NaN,...,NaN,NaN,NaN,400.0,385.0,-105.0,2000.0,800.0,500.0,NaN
4,Gabriel Benitez,Billy Quarantillo,-210,165,47.619048,165.000000,2021-07-17,"Las Vegas, Nevada, USA",USA,NaN,...,NaN,NaN,NaN,140.0,290.0,800.0,1000.0,425.0,475.0,NaN
5,Daniel Rodriguez,Preston Parsons,-265,225,37.735849,225.000000,2021-07-17,"Las Vegas, Nevada, USA",USA,NaN,...,NaN,NaN,NaN,370.0,600.0,NaN,NaN,NaN,NaN,NaN
6,Amanda Lemos,Montserrat Conejo,-500,355,20.000000,355.000000,2021-07-17,"Las Vegas, Nevada, USA",USA,NaN,...,NaN,NaN,NaN,115.0,NaN,700.0,NaN,175.0,NaN,NaN
7,Khalid Taha,Sergey Morozov,-167,135,59.880240,135.000000,2021-07-17,"Las Vegas, Nevada, USA",USA,NaN,...,NaN,NaN,NaN,155.0,225.0,600.0,1200.0,425.0,650.0,NaN
8,Miles Johns,Anderson Dos Santos,-182,145,54.945055,145.000000,2021-07-17,"Las Vegas, Nevada, USA",USA,NaN,...,NaN,NaN,NaN,130.0,310.0,NaN,NaN,NaN,NaN,NaN
9,Francisco Figueiredo,Malcolm Gordon,-305,240,32.786885,240.000000,2021-07-17,"Las Vegas, Nevada, USA",USA,NaN,...,NaN,NaN,NaN,NaN,545.0,NaN,NaN,NaN,NaN,NaN


In [15]:
def display_bets(ev_df, min_ev):
    
    for index, row in ev_df.iterrows():
        t1_bet_ev = get_bet_ev(row['t1_odds'], row['t1_prob'])
        t2_bet_ev = get_bet_ev(row['t2_odds'], row['t2_prob'])
        #print(f"ODDS:{row['t2_odds']} PROB: {row['t2_prob']} EV: {t2_bet_ev}")
        t1_bet_return = get_bet_return(row['t1_odds'])
        t2_bet_return = get_bet_return(row['t2_odds'])
        print(f"{row['t1_name']} vs. {row['t2_name']}")
        if t1_bet_ev > min_ev:
            print(f"{row['t1_name']} has an EV of {round(t1_bet_ev, 2)} on odds of {round(row['t1_odds'],2)}.  They have {round(row['t1_prob']*100,2)}", 
                  "% chance of winning.")
        
        elif t2_bet_ev > min_ev:
            print(f"{row['t2_name']} has an EV of {round(t2_bet_ev, 2)} on odds of {round(row['t2_odds'],2)}.  They have {round(row['t2_prob']*100,2)}", 
                  "% chance of winning.")
        
        else:
            print(f"No bets.  EV1:{t1_bet_ev}. EV2: {t2_bet_ev}")
            print(f"{row['t1_name']} has an EV of {round(t1_bet_ev, 2)} on odds of {round(row['t1_odds'],2)}.  They have {round(row['t1_prob']*100,2)}", 
                  "% chance of winning.")
        
        print()
        
    return(0)
        

In [16]:
def get_bets(input_model, input_features, input_ev, train_df, train_labels, train_odds, test_df, test_labels,
             test_odds, verbose=True):
    fighters_test = df[['R_fighter', 'B_fighter']]
    #Train model.  
    df_train = train_df[input_features].copy()
    df_test = test_df[input_features].copy()
    df_train = df_train.dropna()
    df_test = df_test.dropna()
        
    df_train = pd.get_dummies(df_train)
    df_test = pd.get_dummies(df_test)
    df_train, df_test = df_train.align(df_test, join='left', axis=1)    #Ensures both sets are dummified the same
    df_test = df_test.fillna(0)

    #LOOK AT get_ev and prepare the labels and odds
    
    labels_train = train_labels[train_labels.index.isin(df_train.index)]
    odds_train = train_odds[train_odds.index.isin(df_train.index)] 
    labels_test = test_labels[test_labels.index.isin(df_test.index)]
    odds_test = test_odds[test_odds.index.isin(df_test.index)] 
    fighters_test = fighters_test[fighters_test.index.isin(df_test.index)]
    #print(df_train)
    input_model.fit(df_train, labels_train)
    
    probs = input_model.predict_proba(df_test)
    odds_test = np.array(odds_test)  
    fighters_test = np.array(fighters_test)
    #display(fighters_test)
    prepped_test = list(zip(fighters_test[:, -2], fighters_test[:, -1], odds_test[:, -2], odds_test[:, -1], 
                            probs[:, 0], probs[:, 1], labels_test))
    ev_prepped_df = pd.DataFrame(prepped_test, columns=['t1_name', 't2_name', 't1_odds', 't2_odds', 't1_prob', 't2_prob', 'winner'])
    
    #display(ev_prepped_df)
    
    display_bets(ev_prepped_df, input_ev)
    

    return(ev_prepped_df)

In [17]:
with open('../data/production_models.csv', newline='') as f:
    reader = csv.reader(f)
    models = list(reader)
    
#print(len(models))

#set the model choice automatically:
model_choice = 0
best_score = models[4][0]
for n in range(len(models[0])):
    print(n)
    if (float(models[4][n]) > float((best_score))):
        print("YES")
        best_score = models[4][n]
        model_choice = n

print(f"The model choice is {model_choice}")
print(models)



model = eval(models[1][model_choice])
features = eval(models[2][model_choice])
ev = eval(models[3][model_choice])
print()
print(model, features, ev)

0
1
2
3
4
5
6
7
8
9
10
YES
11
YES
The model choice is 11
[['model_06_13_20', 'model_06_20_20', 'model_6_27_20', 'model_7_18_20', 'model_7_25_20', 'model_9_2_20', 'model_9_10_20', 'model_10_23_20', 'model_10_27_20', 'model_10_28_20', 'model_2021_06_24', 'model_2021_06_30'], ['LogisticRegression(max_iter=50000, random_state=75, tol=5.993308915320019)', 'GaussianNB()', 'GaussianNB()', 'GradientBoostingClassifier(n_estimators=122, random_state=75)', 'GaussianNB()', 'GaussianNB(var_smoothing=1e-12)', 'GaussianNB(var_smoothing=1e-12)', 'GradientBoostingClassifier(max_depth=95, min_samples_leaf=6, n_estimators=195,\n                           random_state=75)', "RandomForestClassifier(criterion='entropy', max_depth=10, n_estimators=197,\n                       n_jobs=-1, random_state=75)", "RandomForestClassifier(criterion='entropy', max_depth=10, n_estimators=22,\n                       n_jobs=-1, random_state=75)", 'GradientBoostingClassifier(max_depth=95, min_samples_leaf=0.0015,\n        

In [18]:

output_df = get_bets(model, features, ev, df_train, label_train, odds_train, df_test, label_test,
                         odds_test, verbose = True)


Islam Makhachev vs. Thiago Moises
Islam Makhachev has an EV of 14.75 on odds of -625.  They have 98.93 % chance of winning.

Marion Reneau vs. Miesha Tate
Marion Reneau has an EV of 104.32 on odds of 105.  They have 99.67 % chance of winning.

Jeremy Stephens vs. Mateusz Gamrot
Jeremy Stephens has an EV of 53.16 on odds of 195.  They have 51.92 % chance of winning.

Rodolfo Vieira vs. Dustin Stoltzfus
Rodolfo Vieira has an EV of 39.4 on odds of -250.  They have 99.57 % chance of winning.

Gabriel Benitez vs. Billy Quarantillo
Gabriel Benitez has an EV of 47.27 on odds of -210.  They have 99.77 % chance of winning.

Amanda Lemos vs. Montserrat Conejo
Amanda Lemos has an EV of 19.33 on odds of -500.  They have 99.44 % chance of winning.

Khalid Taha vs. Sergey Morozov
Sergey Morozov has an EV of 69.55 on odds of 135.  They have 72.15 % chance of winning.

Miles Johns vs. Anderson Dos Santos
Anderson Dos Santos has an EV of 95.89 on odds of 145.  They have 79.95 % chance of winning.

Fran

In [19]:
output_df

,t1_name,t2_name,t1_odds,t2_odds,t1_prob,t2_prob,winner
0,Islam Makhachev,Thiago Moises,-625,425,0.989265,0.010735,NaN
1,Marion Reneau,Miesha Tate,105,-134,0.996674,0.003326,NaN
2,Jeremy Stephens,Mateusz Gamrot,195,-250,0.519170,0.480830,NaN
3,Rodolfo Vieira,Dustin Stoltzfus,-250,195,0.995692,0.004308,NaN
4,Gabriel Benitez,Billy Quarantillo,-210,165,0.997666,0.002334,NaN
5,Amanda Lemos,Montserrat Conejo,-500,355,0.994443,0.005557,NaN
6,Khalid Taha,Sergey Morozov,-167,135,0.278517,0.721483,NaN
7,Miles Johns,Anderson Dos Santos,-182,145,0.200462,0.799538,NaN
8,Francisco Figueiredo,Malcolm Gordon,-305,240,0.992918,0.007082,NaN
9,Alan Baudot,Rodrigo Nascimento,265,-335,0.085064,0.914936,NaN


In [20]:
return_df = output_df[['t1_name', 't2_name', 't1_prob', 't2_prob']]

In [21]:
return_df

,t1_name,t2_name,t1_prob,t2_prob
0,Islam Makhachev,Thiago Moises,0.989265,0.010735
1,Marion Reneau,Miesha Tate,0.996674,0.003326
2,Jeremy Stephens,Mateusz Gamrot,0.519170,0.480830
3,Rodolfo Vieira,Dustin Stoltzfus,0.995692,0.004308
4,Gabriel Benitez,Billy Quarantillo,0.997666,0.002334
5,Amanda Lemos,Montserrat Conejo,0.994443,0.005557
6,Khalid Taha,Sergey Morozov,0.278517,0.721483
7,Miles Johns,Anderson Dos Santos,0.200462,0.799538
8,Francisco Figueiredo,Malcolm Gordon,0.992918,0.007082
9,Alan Baudot,Rodrigo Nascimento,0.085064,0.914936


In [22]:
output_df = return_df.rename(columns = {'t1_name': 'R_fighter', 't2_name': 'B_fighter', 't1_prob': 'R_prob', 't2_prob': 'B_prob'})

In [23]:
output_df.to_csv('../data/upcoming_probs.csv', index=False)